In [ ]:
from natsort import natsorted
from glob import glob
from itertools import product
import os
from pprint import pprint

import torch
import numpy as np
from umap import UMAP
# from sklearn.decomposition import PCA

from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from matplotlib import colormaps

import data_utils

plt.style.use("ggplot")
plt.style.use("seaborn-v0_8-colorblind")

In [ ]:
# ## Load vocabulary

# # with open("./data/20k.txt") as f:
# # with open("./data/nouns_and_adjectives.txt") as f:
# with open("./data/wordnet_hierarchy.txt") as f:
#     vocabulary = [v.strip() for v in f]
# ", ".join(vocabulary[:20])

## Load Data

In [ ]:
natsorted(glob('my_data/*/'))

In [ ]:
## Choose the models to jointly (all layers in all models) compute UMAP on

dir_in = 'my_data'

# models = []
# models += natsorted(glob('my_data/resnet*/'))
# models += natsorted(glob('my_data/vit*/'))
# models += natsorted(glob('my_data/googlenet*/'))
# models += natsorted(glob('my_data/*attack*/'))
# model_groups_to_compare = [models]

model_groups_to_compare  = [
    ['architectures', ['resnet34', 'resnet50', 'googlenet', 'vit_b_16']],
    ['splits', ['resnet50_split0', 'resnet50_split1']],
    ['artificial_vs_natural', ['resnet50_artificial', 'resnet50_natural']],
    ['attacks', ['resnet50_under_attack', 'resnet50robust_under_attack']],
]

model_groups_to_compare = [
    [group_name, [f'{dir_in}/{m}/' for m in models]]
    for group_name, models in model_groups_to_compare
]

pprint(model_groups_to_compare)

In [ ]:
def center_and_rotate(xy):
    xy = xy - xy.mean(axis=0)
    best_rot = None
    largest_aspect_ratio = -1
    for angle in np.linspace(0, np.pi/2, 14):
        rot = np.array([
            [np.cos(angle), np.sin(angle)], 
            [-np.sin(angle), np.cos(angle)]
        ])
        xy_rot = xy @ rot
        w = xy_rot[:,0].max() - xy_rot[:,0].min()
        h = xy_rot[:,1].max() - xy_rot[:,1].min()
        aspect_ratio = w/h
        if aspect_ratio > largest_aspect_ratio:
            largest_aspect_ratio = aspect_ratio
            best_rot = rot
    xy = xy @ best_rot
    return xy

In [ ]:
dir_out = 'my_data/neuron_umaps'
os.makedirs(dir_out, exist_ok=True)

for group_name, model_group in tqdm(model_groups_to_compare):
    ## load data
    model_layer_neurons = []
    sim_data = []
    for m in model_group:
        sim = torch.load(f"./{m}/all_layer_similarities.pt")
        m = m.split('/')[-2]
        model_layer_neurons += [[m, s['layer'], s['similarities'].shape[0]] for s in sim]
        sim_data += [s['similarities'] for s in sim]
    sim_data = torch.cat(sim_data)
    sim_data = sim_data.numpy()
    
    print(
        'Computing UMAP of group', model_group, '\n',
        'data shape', sim_data.shape, '\n',
        'layers', model_layer_neurons[:2], '...'
    )
    
    xy = UMAP(
        n_neighbors=300,
        min_dist=0.8, 
        n_components=2,
    ).fit_transform(sim_data)
    xy = center_and_rotate(xy)
    
#     xy = PCA().fit_transform(xy)
#     ## PCA makes array in fortran order, turning in back to "C order" :((
#     xy = np.ascontiguousarray(xy) 

    plt.scatter(
        xy[:,0], 
        xy[:,1], 
        s=0.5,
    )
    plt.axis('equal')
    
    os.makedirs(f"{dir_out}/{group_name}", exist_ok=True)
    plt.savefig(f"{dir_out}/{group_name}/plot.png", dpi=200, bbox_inches='tight')
    plt.show()
    
    
    ## vis/save umap to file
    vis = False
    start = 0
    n_neurons_per_layer = [s[2] for s in model_layer_neurons]
    for start,s in zip([0,*np.cumsum(n_neurons_per_layer)],model_layer_neurons) :
        
        n = s[2]
        xy_layer = xy[start:start+n]
        
        fn = f'{s[0]}_{s[1]}'
        os.makedirs(f"{dir_out}/{group_name}", exist_ok=True)
        out_filepath = f"{dir_out}/{group_name}/{fn}.npy"
        np.save(out_filepath, xy_layer.astype(np.float16))
        print('save umap in:', out_filepath)
        
        if vis:
            plt.figure(figsize=[3,3])
            plt.scatter(
                xy_layer[:,0], 
                xy_layer[:,1], 
                s=1,
                label=f'{s[0]}, {s[1]}'
            )
            plt.legend()
            plt.axis('equal')
            plt.show()
        

In [ ]:
# # TODO: rotate:
# # 1. center
# # 2. for angle in angles, rotate data
# # .compute aspect ratio
# # keep best angle and largest aspect ratio

# def center_and_rotate(xy):
#     xy = xy - xy.mean(axis=0)
#     best_rot = None
#     largest_aspect_ratio = -1
#     for angle in np.linspace(0, np.pi/2, 14):
#         rot = np.array([
#             [np.cos(angle), np.sin(angle)], 
#             [-np.sin(angle), np.cos(angle)]
#         ])
#         xy_rot = xy @ rot
#         w = xy_rot[:,0].max() - xy_rot[:,0].min()
#         h = xy_rot[:,1].max() - xy_rot[:,1].min()
#         aspect_ratio = w/h
#         if aspect_ratio > largest_aspect_ratio:
#             largest_aspect_ratio = aspect_ratio
#             best_rot = rot
#     xy = xy @ best_rot
#     return xy

# plt.scatter(
#     xy[:,0], 
#     xy[:,1], 
#     s=0.5,
# )
# plt.axis('equal')
# plt.show()

In [ ]:
# plt.scatter(
#     xy[:,0], 
#     xy[:,1], 
#     s=0.1,
# )
# plt.axis('equal')
# plt.show()

In [ ]:
# c = torch.cat([torch.ones(s[2])+i for i,s in enumerate(model_layer_neurons)]).numpy()
# s = torch.cat([torch.ones(s[2])+i for i,s in enumerate(model_layer_neurons)]).numpy()


---

In [ ]:
# ## UMAP by activation

# # layer1 = torch.load('./my_data/resnet50_under_attack/imagenet_val_attack_resnet50_layer4.pt')
# # layer2 = torch.load('./my_data/resnet50robust_under_attack/imagenet_val_attack_resnet50robust_layer4.pt')

# # layer1 = torch.load('./my_data/resnet50/imagenet_val_resnet50_layer4.pt')
# # layer2 = torch.load('./my_data/resnet50robust/imagenet_val_resnet50robust_layer4.pt')

# layer1 = torch.load('./my_data/resnet34/imagenet_val_resnet34_layer4.pt')
# layer2 = torch.load('./my_data/resnet50/imagenet_val_resnet50_layer4.pt')


In [ ]:
# umap_data = torch.cat([layer1.t(), layer2.t()]).cpu().numpy()
# umap_data.shape

In [ ]:
# umap = UMAP(n_neighbors=100).fit_transform(umap_data)

In [ ]:
# c=torch.concat([
#     torch.zeros(layer1.shape[1]), 
#     torch.ones(layer2.shape[1])
# ])

In [ ]:
# plt.scatter(
#     umap[:,0], 
#     umap[:,1], 
#     s=5,
#     c=c,
#     cmap='tab10'
# )

In [ ]:
# layer-pairwise UMAP
# umaps = []
# for m1, m2 in product(models, models):
#     if m1>m2:
#         continue
                
#     ## Load neuron-concept similarity
#     sim1 = torch.load(f"./{m1}/all_layer_similarities.pt")
#     sim2 = torch.load(f"./{m2}/all_layer_similarities.pt")
#     layer_names1 = [s['layer'] for s in sim1]
#     layer_names2 = [s['layer'] for s in sim2]
    
#     m1, m2 = m1.split('/')[-2], m2.split('/')[-2]
#     for l1, l2 in product(
#         range(len(layer_names1)), 
#         range(len(layer_names2))
#     ):
#         if l1>l2: 
#             continue
        
#         data = torch.cat([sim1[l1]["similarities"],sim2[l2]["similarities"]]).numpy()
#         print(data.shape)
#         xy = UMAP(n_components=2, min_dist=0.3).fit_transform(data)
#         ## num neurons
#         n1 = sim1[l1]["similarities"].shape[0]
#         n2 = sim2[l2]["similarities"].shape[0]
#         plt.figure(figsize=[4,4])
#         plt.scatter(
#             xy[:,0], 
#             xy[:,1], 
#             s=torch.cat([10+torch.zeros(n1), 5+torch.zeros(n2)]), 
#             c=torch.cat([torch.zeros(n1), torch.ones(n2)]), 
#             cmap='tab10'
#         )
#         plt.title([m1, m2, layer_names1[l1], layer_names2[l2]])
#         plt.show()
        
#         umaps.append([m1, m2, layer_names1[l1], layer_names2[l2], xy])
#         umaps.append([m2, m1, layer_names2[l2], layer_names1[l1], xy])
        

### Neuron UMAP, Each individual layers

In [ ]:
# for i,[m1, m2, l1, l2, umap] in enumerate(umaps):
#     plt.figure()
#     plt.scatter(
#         umap[:, 0],
#         umap[:, 1],
#         s=10,
#         linewidth=0.1,
#         edgecolors="#333",
#     )
#     plt.axis("equal")
#     plt.legend()
#     plt.show()

## Save UMAP to file

In [ ]:

# for i,[m1, m2, l1, l2, umap] in enumerate(umaps):
#     fn = f'{m1}_{l1}_{m2}_{l2}'
#     np.save(f"{dir_out}/{fn}.npy", umap)
#     print(f"{dir_out}/{fn}.npy")

---

In [ ]:
# #TODO? SpectralCoclustering

# from sklearn.cluster import SpectralCoclustering

# clustering = SpectralCoclustering().fit(im)

# row = np.argsort(clustering.row_labels_)
# col = np.argsort(clustering.column_labels_)


## Save neuron top-n concept indices to file

In [ ]:
# for sim in similarity_load:
#     layer_name = sim['layer']
#     sim = sim['similarities'].argsort(descending=True)
#     sim = sim[:,:100] ## get top
#     sim = sim.type(torch.int32)
#     fn = f'{dir_out}/concepts_top100_{layer_name}.npy'
#     np.save(fn, sim.numpy())

## Copy vocabulary file

In [ ]:
# !cp data/20k.txt {dir_out}/vocabulary_20k.txt